In [ ]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator

import pandas as pd
import numpy as np
import os
from os.path import join
import re
import ast
import sys
#from Evaluator import Evaluator
import src
import data_patterns

In [ ]:
RULES_PATH = join('..', 'solvency2-rules')
FILENAME_RULES = '2020-01-22 Set aanvullende controleregels Solvency II_tcm46-387021.xlsx'

## Construct test Solvency 2 instance (put here your own data)

In [ ]:
RESULTS_PATH = join('..', 'results')
DATA_PATH = join('..', 'data')

In [ ]:
df = pd.DataFrame()
df.index.name = "index"
files = [f for f in os.listdir(RESULTS_PATH) if os.path.isfile(join(RESULTS_PATH, f)) if f[-6:]=='pickle']
for file in files:
    new_df = pd.read_pickle(join(RESULTS_PATH, file))
    new_df.columns = [col.upper() for col in new_df.columns]
    if list(new_df.index) == [0]: # without z-axis for now
        for col in new_df.columns:
            if col not in df.columns:
                df[col] = new_df[col]
    else:
        for col in new_df.columns:
            if col not in df.columns:
                # we only pick the first line in the z-axis column
                df.loc[0, col] = new_df.iloc[0, new_df.columns.get_loc(col)]
        df.loc[0, new_df.index.name] = new_df.index[0]
df = df.astype(object)

In [ ]:
df.head()

## Create simple taxonomy based on instance

In [ ]:
df_taxo = pd.DataFrame(columns = ['datapoint', 'template', 'row', 'column', 'dtype'])
for idx, col in enumerate(df.columns):
    df_taxo.loc[idx, "datapoint"] = col.upper()
    df_taxo.loc[idx, "template"] = col[0:13].upper()
    df_taxo.loc[idx, "row"] = col[14:19].upper()
    df_taxo.loc[idx, "column"] = col[20:25].upper()
    df_taxo.loc[idx, "dtype"] = df.dtypes[idx]
df_taxo.head(5)

# for now we only use the list of templates in the instance
instance_templates = list(df_taxo.loc[:, 'template'].unique())
del df_taxo

## Read DNBs Additional Validation Rules

In [ ]:
df_rules = pd.read_excel(join(RULES_PATH, FILENAME_RULES), header = 1)
df_rules = df_rules.set_index('ControleRegelCode')
df_rules = df_rules.drop('S.28.01_129', axis = 0) # double line, should be removed
df_rules = df_rules.drop('S.01.03_110', axis = 0) # double line, should be removed
df_rules.fillna("", inplace = True)

## Parse formulas

In [ ]:
evaluator = src.evaluator.Evaluator(df, df_rules, instance_templates)

In [ ]:
evaluator.transform_rules()

In [ ]:
evaluator.evaluate_rules()

In [ ]:
miner = data_patterns.PatternMiner(df_patterns = evaluator.df_patterns)
results = miner.analyze(df)

In [ ]:
miner.df_patterns[miner.df_patterns['Error message']!='']

In [ ]:
results[results['Q values'] == 'BUG']